# Agents For Questioning Napoleon and Alexander

This notebook demonstrates multiple agentic queries of both the Napoleon and Alexander movies Wikipedia pages, as well as the corresponding web scrapped IMDB movie reviews. The IMDB review scraping was performed independently of this notebook.

## Build

This notebook:

1. Contructs a Napoleon retriever.
2. Constructs an Alexander retriever.
3. Combines the two and allow users to agentically query both resources from a single input.

### Install Libraries

In [47]:
# Install dependencies:
import warnings
warnings.filterwarnings('ignore')

%pip install --upgrade --quiet langchain==0.2.0
!pip install langchain_community==0.2.1
!pip install -U langchain-openai==0.1.8
!pip install -q -U wikipedia==1.4.0
!pip install -q -U rank_bm25==0.2.2 tiktoken==0.7.0 faiss-gpu==1.7.2
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain.document_loaders import WikipediaLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.prompts import ChatPromptTemplate
from langchain.agents import Tool
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.chains import LLMChain
import pandas as pd
import tensorflow as tf

In [48]:
# Remove limit on column display width
pd.options.display.max_colwidth = None

In [49]:
# Utilize GPU if available
# Get the list of available physical devices
physical_devices = tf.config.list_physical_devices('GPU')

if len(physical_devices) > 0:
    # If a GPU is available, use it
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    device = "/GPU:0"
    print("Using GPU")
else:
    # If no GPU is available, use CPU
    device = "/CPU:0"
    print("Using CPU")

Using GPU


In [50]:
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API Key: ··········


### LLM

OpenAI's `gpt-4`is utilized.

In [51]:
# Define llm
llm = ChatOpenAI(model="gpt-4", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia.

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [52]:
# Definition of Napoleon document retrieval and csv loader
# from langchain.document_loaders import WikipediaLoader, CSVLoader

napoleon_wikipedia_docs = WikipediaLoader(
    query="Napoleon (2023 film)",
    load_max_docs = 1,
    doc_content_chars_max = 1_000_000
    ).load()

# Load csv file that was previously scrapped from the IMDB movie review website
napoleon_csv_docs = CSVLoader(
    file_path = "/content/napoleon.csv",
    source_column = "Review_Url"
    ).load()

In [53]:
# Import recursive character text splitter
# from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0,
    length_function = len,
    is_separator_regex= False,
    separators = ["\n==", "\n", " "] # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    length_function = len,
    is_separator_regex = False,
    separators = ["\n", " "] # keep paragraphs, then sentences
)

chunked_napoleon_wikipedia_docs = wikipedia_text_splitter.transform_documents(napoleon_wikipedia_docs)
chunked_napoleon_csv_docs = csv_text_splitter.transform_documents(napoleon_csv_docs)

#### Retrieval and Embedding

In [54]:
# set up cached embeddings store
store = LocalFileStore("./shared_cache/")

# Select embedding model
core_embeddings_model = OpenAIEmbeddings()

# Select cached embedding
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=core_embeddings_model.model
)


In [55]:
# Implement FAISS vectorsstore and create retriever
napoleon_csv_faiss_retriever = FAISS.from_documents(chunked_napoleon_csv_docs, embedder).as_retriever()

### Create Ensemble Retriever

The process is:

1. Collect results from two different retrieval methods over the same corpus
2. Apply a reranking algorithm to rerank source documents to select the most relevant without losing specific or potentially low-ranked information rich documents
3. Feed the top-k results into the LLM with the query as context.

In [56]:
# Set up BM25 retriever
napoleon_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_napoleon_wikipedia_docs
)

napoleon_wikipedia_bm25_retriever.k = 1

# Set up FAISS vector store
napoleon_wikipedia_faiss_store = FAISS.from_documents(
    chunked_napoleon_wikipedia_docs,
    embedder
)

napoleon_wikipedia_faiss_retriever = napoleon_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# Set up ensemble retriever with 75% weighting to FAISS retriever and 25% weighting to BM25 retriever
napoleon_ensemble_retriever = EnsembleRetriever(
    retrievers = [napoleon_wikipedia_bm25_retriever, napoleon_wikipedia_faiss_retriever],
    weights = [0.25, 0.75]
)

#### Create Retrieval Tools and Agents

In [57]:
# Create simple conversational retrieval tools

# Define wikipedia retrieval tool
napoleon_wikipedia_retrieval_tool = create_retriever_tool(
    napoleon_ensemble_retriever,
    "Wikipedia",
    "Searches and returns documents regarding the plot, history, and cast of the Napoleon movie"
)

# Define csv retrieval tool
napoleon_csv_retrieval_tool = create_retriever_tool(
     napoleon_csv_faiss_retriever,
    "PublicReviews",
    "Searches and returns documents regarding public reviews of the Napoleon movie"
)

# Define Napoleon_retrieval_tools list
napoleon_retriever_tools = [napoleon_wikipedia_retrieval_tool, napoleon_csv_retrieval_tool]

In [58]:
# Combine Napoleon retrieval tools into an agent
napoleon_retriever_agent_executor = create_conversational_retrieval_agent(llm, napoleon_retriever_tools, verbose=True)

In [59]:
# Query Napoleon retriever
napoleon_retriever_agent_executor({"input" : "Were there comments about whether Napoleon was a historically accurate movie? If there were, what was the sentiment about the historical accuracy?"})



> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `{'query': 'historical accuracy of Napoleon movie'}`


Review_Date: 11-Dec-23
Author: apereztenessa-1
Rating: 7
Review_Title: Innovative in its way to depict a historic legend
Review: Napoleon has been a surprise hit, and how much ink has been spent on the film may be one reason for its success. Rarely in recent history has a film been so widely criticized for its historical inaccuracies. We didn't hear that kind of backlash for The Crown or Bridgerton, so why such disappointment in historic representation in this specific case?
Review_Url: /review/rw9450470/?ref_=tt_urv

Review: This should have been called "Napoleon and Josephine" because, frankly, there's too much of Josephine in it, and not nearly enough of the brilliance and personality of Bonaparte. The historical inaccuracies are manifold. I read that director Scott says that "If you weren't there then you can **** off". Well I was not there, but the erroneo

{'input': 'Were there comments about whether Napoleon was a historically accurate movie? If there were, what was the sentiment about the historical accuracy?',
 'chat_history': [HumanMessage(content='Were there comments about whether Napoleon was a historically accurate movie? If there were, what was the sentiment about the historical accuracy?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "historical accuracy of Napoleon movie"\n}', 'name': 'PublicReviews'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 151, 'total_tokens': 171}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-ded8c6d6-2dee-4bb5-a48c-9911a567d81f-0', usage_metadata={'input_tokens': 151, 'output_tokens': 20, 'total_tokens': 171}),
  FunctionMessage(content='Review_Date: 11-Dec-23\nAuthor: apereztenessa-1\nRating: 7\nReview_Title: Innovative in its way to depict a historic legend\

In [60]:
# Query Napoleon retriever
napoleon_retriever_agent_executor({"input" : "What is a quick summary of the plot of the Napoleon  movie?"})



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `{'query': 'plot of Napoleon movie'}`


Napoleon is a 2023 epic historical drama film directed and co-produced by Ridley Scott and written by David Scarpa. Based on the story of Napoleon Bonaparte and primarily depicting his rise to power as well as his relationship with his wife, Joséphine Bonaparte, it stars Joaquin Phoenix as Napoleon and Vanessa Kirby as Joséphine.

Joan Tumblety wrote that the movie ignored French atrocities such as those committed after the Siege of Jaffa.The Napoleon movie, directed by Ridley Scott and written by David Scarpa, is an epic historical drama film released in 2023. The film primarily depicts the rise to power of Napoleon Bonaparte and his relationship with his wife, Joséphine Bonaparte. The lead roles are played by Joaquin Phoenix as Napoleon and Vanessa Kirby as Joséphine. The movie has been criticized for ignoring certain historical events, such as the French atrocities committed a

{'input': 'What is a quick summary of the plot of the Napoleon  movie?',
 'chat_history': [HumanMessage(content='Were there comments about whether Napoleon was a historically accurate movie? If there were, what was the sentiment about the historical accuracy?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "historical accuracy of Napoleon movie"\n}', 'name': 'PublicReviews'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 151, 'total_tokens': 171}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-ded8c6d6-2dee-4bb5-a48c-9911a567d81f-0', usage_metadata={'input_tokens': 151, 'output_tokens': 20, 'total_tokens': 171}),
  FunctionMessage(content='Review_Date: 11-Dec-23\nAuthor: apereztenessa-1\nRating: 7\nReview_Title: Innovative in its way to depict a historic legend\nReview: Napoleon has been a surprise hit, and how much ink has been spent on the film 

### Alexander Retrieval System

Some of previously created code will be repurposed, also a different multi-source retrieval system will be built

In [61]:
# Define Alexander document loaders

# Load wikipedia data
alexander_wikipedia_docs = WikipediaLoader(query="Alexander (2004 film)", load_max_docs=1, doc_content_chars_max=1_000_000).load()

# Load csv data previously scrapped from IMDB reviews of the Alexander movie
alexander_csv_docs = CSVLoader(file_path="./alexander.csv", source_column="Review_Url").load()

In [62]:
# Perform text chunking
chunked_alexander_wikipedia_docs = wikipedia_text_splitter.transform_documents(alexander_wikipedia_docs)
chunked_alexander_csv_docs = csv_text_splitter.transform_documents(alexander_csv_docs)

In [63]:
# Define retrievers for Alexander
alexander_csv_faiss_retriever = FAISS.from_documents(chunked_alexander_csv_docs, embedder).as_retriever()

# Set up BM25 retriever
alexander_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_alexander_wikipedia_docs
)

alexander_wikipedia_bm25_retriever.k = 1

# Set up FAISS vector store
alexander_wikipedia_faiss_store = FAISS.from_documents(
    chunked_alexander_wikipedia_docs, embedder
)

# Set up Alexander retriever
alexander_wikipedia_faiss_retriever = alexander_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# Set up ensemble retriever
alexander_ensemble_retriever = EnsembleRetriever(
    retrievers=[alexander_wikipedia_bm25_retriever, alexander_wikipedia_faiss_retriever],
    weights=[0.25, 0.75]
)

#### Multi-source chain

The LLM will be used to determine which information is most -> least valuable.

In [64]:
# from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Alexander movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Alexander movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [65]:
alexander_multisource_chain = {
    "source1": (lambda x: x["question"]) | alexander_ensemble_retriever,
    "source2": (lambda x: x["question"]) | alexander_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [66]:
alexander_multisource_chain.invoke({"question" : "What did people think of the Alexander movie?"})

AIMessage(content="The reviews for the Alexander movie were mixed. Some viewers appreciated the film for its historical content and portrayal of Alexander the Great's life. They praised the acting, particularly Colin Farrell's performance as Alexander, and the ambitious scope of the film. They also enjoyed the battle sequences and the film's attention to historical detail.\n\nHowever, there were also several criticisms. Some viewers felt that the film was too long and had pacing issues. They also thought that the narrative was meandering and lacked focus. Some critics, including Roger Ebert, felt that the film resembled a history documentary more than an action-drama film. The casting choices were also seen as a mix of hits and misses. The film was also criticized for its historical accuracy. Overall, while some viewers found the film to be a worthwhile watch, others felt that it had significant weaknesses.", response_metadata={'token_usage': {'completion_tokens': 168, 'prompt_tokens':

# Agent Creation

The Agent will be provided a Toolbelt (list of tools).

A function will be set for tools to interface with the Agent.

In [67]:
# Define query
def query_alexander(input):
    return alexander_multisource_chain.invoke({"question" : input})

tools = [
    Tool(
        name = "napoleoninfo",
        func = napoleon_retriever_agent_executor.invoke,
        description = "Useful for when you need to answer questions about Napoleon. Input should be a fully formed question."
    ),

    Tool(
        name = "alexanderinfo",
        func = query_alexander,
        description = "useful for when you need to answer questions about Alexander. Input should be a fully formed question."
    ),
]

### Perform Zero Shot Query

In [68]:
# Configure prompt

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
     tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "agent_scratchpad"]
)

In [69]:
from langchain.chains import LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [70]:
napoleonalexander_agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

In [71]:
napoleonalexander_agent_chain = AgentExecutor.from_agent_and_tools(agent=napoleonalexander_agent, tools=tools,verbose=True)


## Conclusion

Feed inputs to the Agent and review the outputs.

In [72]:
napoleonalexander_agent_chain.invoke({"input" : "What do people today think about Napoleon?"})



> Entering new AgentExecutor chain...
Thought: This question is about Napoleon, so I should use the napoleoninfo tool.
Action: napoleoninfo
Action Input: {"input": "What do people today think about Napoleon?"}

> Entering new AgentExecutor chain...

Invoking: `PublicReviews` with `{'query': 'public opinion on Napoleon movie'}`


Review_Date: 9-Dec-23
Author: adambarta
Rating: 6
Review_Title: Hollywood overtaking over quality
Review: Once I found out that there will be a movie about Napoleon starring Joaquin Phoenix, I got excited. What a great idea? It's one of the most interesting people in history, of course this would make a great movie.
Review_Url: /review/rw9450470/?ref_=tt_urv

Review_Date: 26-Nov-23
Author: krzysiektom
Rating: 2
Review_Title: Awful.
Review: What a disappointment. This film is BORING. A movie with a huge budget, good actors, about a person as fascinating and complex as Bonaparte, who participated in more than 60 battles and had drama, triumphs and failures that

{'input': 'What do people today think about Napoleon?',
 'output': 'Public opinion on the Napoleon movie is mixed. Some viewers were excited about the idea of a movie about Napoleon, especially with Joaquin Phoenix in the lead role. They found Napoleon to be one of the most interesting figures in history and were looking forward to seeing his story on the big screen.\n\nHowever, many viewers were disappointed with the film. They found it boring and felt that it did not do justice to the fascinating and complex character of Napoleon. They criticized the film for not explaining things well, for having unnecessary sex scenes, and for not making it clear why the French nation would follow and worship Napoleon as he was portrayed in the movie. They also felt that the movie was visually depressing, with a greyish-blue color palette.\n\nSome viewers also felt that the movie did not have a clear focus and lacked cinematic flow, resulting in a confusing viewing experience. They felt that making

In [73]:
napoleonalexander_agent_chain.invoke({"input" : "What do people today think about Alexander?"})



> Entering new AgentExecutor chain...
Thought: The question is about Alexander, so I should use the alexanderinfo tool.
Action: alexanderinfo
Action Input: What do people today think about Alexander?
Observation: content='The reviews for the movie "Alexander" are mixed. On Rotten Tomatoes, the film holds an approval rating of 16% based on 206 reviews, with an average rating of 4.00/10. The website\'s critical consensus states that the film is ponderous, talky, and emotionally distant. On Metacritic, the film has a weighted average score of 40 out of 100, based on 42 critics, indicating "mixed or average reviews". Audiences polled by CinemaScore gave the film an average grade of "D+" on an A+ to F scale. Some user reviews praise the film for its historical accuracy and performances, particularly those of Angelina Jolie and Colin Farrell, while others criticize it for being too long and flawed across many aspects.' response_metadata={'token_usage': {'completion_tokens': 160, 'prompt_to

{'input': 'What do people today think about Alexander?',
 'output': 'The general public opinion about Alexander the Great is often positive. He is widely regarded as one of the greatest military tacticians and strategists in history, and he is often celebrated for his tactical skills and his significant impact on spreading Greek culture and civilization across a vast empire. However, some people criticize him for his methods and the destruction caused during his conquests. Please note that public opinion can vary greatly depending on cultural, historical, and personal perspectives.'}

In [74]:
napoleonalexander_agent_chain.invoke({"input" : "Did the movies Napoleon and Alexander share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: This question is asking about the themes or ideas in the movies Napoleon and Alexander. I don't have direct information about the movies, but I can provide information about Napoleon and Alexander themselves, which might help infer the themes of the movies. 

Action: napoleoninfo
Action Input: What are some key themes or ideas associated with Napoleon?


> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `{'query': 'themes and ideas associated with Napoleon Bonaparte'}`


Napoleon was known for his passion for literature and Enlightenment thought – a far cry from the boorish, loutish soldier that Scott depicts. French historian Patrice Gueniffey called the movie "anti-French" and full of historical inaccuracies, while fellow historian Romain Marsily found the portrayal of Napoleon "lackluster" and “mediocre” while disregarding Napoleon's positive legacies such as the Napoleonic Code. Spanish historian Francisco Gracia Alon

{'input': 'Did the movies Napoleon and Alexander share similar themes or ideas?',
 'output': "The key themes or ideas associated with Napoleon include Enlightenment Thought, his rise to power, the Napoleonic Code, and his relationship with Joséphine. For Alexander, the key themes include love, honor, music, exploration, poetry, military combat, and his relationships with his mother Olympias, his tutor Aristotle, and his father, Philip II of Macedon. \n\nWhile both figures are associated with themes of power and relationships, the specific themes and ideas associated with each are quite distinct. Napoleon is more associated with political and legal reforms and Enlightenment thought, while Alexander is associated with themes of love, honor, music, and exploration. Therefore, it's not clear that the movies Napoleon and Alexander would share similar themes or ideas based on the themes associated with the historical figures themselves."}

**Conclusion**

* This concludes the examples of LangChain based Agentic Retrieval and Question Answering (RAQA)